In [18]:
import numpy as np
import os
import cv2
import pandas as pd
import pickle

In [19]:
# Face detection model:
face_detection_model_path = "./moudules/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# Mô tả kiến trúc:
face_detection_proto_path = "./moudules/deploy.prototxt"

# Mô hình mô tả đặc trưng khuôn mặt:
face_descriptor_path = "./moudules/openface.nn4.small2.v1.t7"

In [20]:
# Sử dụng OpenCV DNN đọc mô hình nhận dạng khuôn mặt:
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto_path, face_detection_model_path)

# Đọc mô hình mô tả đặc trưng khuông mặt từ file Torch:
description_model = cv2.dnn.readNetFromTorch(face_descriptor_path)

In [21]:
# Danh sách hình ảnh chuẩn bị
faces = "./img/01/"
os.listdir(faces)

['01.jpg',
 '02.jpg',
 '03.jpg',
 'animals.jpg',
 'avata.jpg',
 'Capture.PNG',
 'cat.jpg',
 'face.jpg',
 'face01.jpg',
 'face02.jpg',
 'face03.jpg',
 'face04.jpg',
 'face1.png',
 'gray_img.PNG',
 'musk01.jpg',
 'musk02.jpg',
 'musk03.jpg',
 'musk04.jpg',
 'musk05.jpg',
 'musk06.jpg',
 'musk07.jpg',
 'musk08.jpg',
 'musk09.jpg',
 'musk10.jpg',
 'musk11.jpg',
 'new_01.JPG',
 'new_01.PNG',
 'thing.jpg',
 'videocat.mp4']

In [22]:
# Hiển thị 1 ảnh:
imgMusk = "img/01/musk11.jpg"
img_orin = cv2.imread(imgMusk)
cv2.imshow("musk",img_orin)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
# copy ảnh:
img = img_orin.copy()

# Lấy chiều cao rộng của ảnh:
h, w = img.shape[:2]

# Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt:
imgBlob = cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB = False, crop = False)

# Thiết lập đầu vào cho mô hình:
detector_model.setInput(imgBlob)

# Nhận diện khuôn mặt:
detections = detector_model.forward()

# Kiểm tra xem có nhận diện được khuôn mặt không:
if(len(detections)>0):
    # Chọn khuôn mặt có độ tinn cậy cao nhất:
    i = np.argmax(detections[0,0,:,2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng -> confidence
    confidence = detections[0,0,i,2]

    # Kiểm tra độ tin cậy cao hơn 0.5
    if(confidence > 0.5):
        box = detections[0,0,i,3:7] * np.array([w,h,w,h])
        (startx, starty, endx, endy) = box.astype("int")

        # Trích xuất vùng ảnh chauw khuôn mặt:
        region_img = img[starty : endy, startx : endx]
        #cv2.imshow("hehehe",region_img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        # Chuẩn bị dữu liệu đầu vào cho mô hình trích xuất đặc trưng:
        faceBlob = cv2.dnn.blobFromImage(region_img,1/255,(96,96),(0,0,0),swapRB = True,crop = True)

        # Thiết lập mô hình cho đầu vào
        description_model.setInput(faceBlob)

        # Trích xuất đặc trưng ra:
        vectors = description_model.forward()

        print(vectors)
        

[[ 0.04640768  0.08532926  0.05444498 -0.03076323 -0.04105002  0.1753755
  -0.02763496  0.08523064  0.03065675 -0.04221327  0.15991545  0.08758998
   0.03920537 -0.17085832  0.00917696 -0.09094014 -0.09081321  0.06495679
  -0.07393188  0.02526897  0.14279366  0.11093555 -0.0168124   0.07454357
  -0.00501479 -0.04971514  0.06572007 -0.116075   -0.0161467   0.09293649
   0.02780767 -0.16560438  0.10308194  0.08295382  0.03798639 -0.03754817
   0.01706891 -0.00399489  0.00351112  0.0558585   0.023383   -0.08413078
  -0.05833076  0.1410759  -0.14615469 -0.17002092  0.06162611 -0.1116887
  -0.10542265  0.08228737  0.09269734  0.02662694 -0.0891658   0.08061194
  -0.00581358  0.06398202 -0.06902802 -0.01796903 -0.12903333 -0.07384918
  -0.12604652 -0.16539364  0.1308393  -0.08341269  0.03384787 -0.1783063
   0.06822301 -0.05200874 -0.08646934  0.06762562  0.07753393 -0.05037301
  -0.05495495 -0.12944992  0.03722917  0.02161354 -0.11069066  0.01227776
  -0.02402477  0.16864523 -0.08670732 -0.

In [48]:
# Tạo function để áp dụng cho nhiều ảnh:
def detect_imgs(img_path):
    img_orin = cv2.imread(img_path)
    
    # copy ảnh:
    img = img_orin.copy()
    
    # Lấy chiều cao rộng của ảnh:
    h, w = img.shape[:2]
    
    # Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt:
    imgBlob = cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB = False, crop = False)
    
    # Thiết lập đầu vào cho mô hình:
    detector_model.setInput(imgBlob)
    
    # Nhận diện khuôn mặt:
    detections = detector_model.forward()
    
    # Kiểm tra xem có nhận diện được khuôn mặt không:
    if(len(detections)>0):
        # Chọn khuôn mặt có độ tinn cậy cao nhất:
        i = np.argmax(detections[0,0,:,2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng -> confidence
        confidence = detections[0,0,i,2]
    
        # Kiểm tra độ tin cậy cao hơn 0.5
        if(confidence > 0.5):
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            (startx, starty, endx, endy) = box.astype("int")
    
            # Trích xuất vùng ảnh chauw khuôn mặt:
            region_img = img[starty : endy, startx : endx]
            #cv2.imshow("hehehe",region_img)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
    
            # Chuẩn bị dữu liệu đầu vào cho mô hình trích xuất đặc trưng:
            faceBlob = cv2.dnn.blobFromImage(region_img,1/255,(96,96),(0,0,0),swapRB = True,crop = True)
    
            # Thiết lập mô hình cho đầu vào
            description_model.setInput(faceBlob)
    
            # Trích xuất đặc trưng ra:
            vectors = description_model.forward()
    
            return vectors
        

In [49]:
# Áp dụng cho các ảnh:

# Khởi tạo dictionary:
data = dict(data=[], label=[])
mydir = "./img/01/train"
folders = os.listdir(mydir)

# Duyệt qua từng thư mục:
for folder in folders:
    path = mydir + "/" + folder
    files = os.listdir(path)

    for fileName in files:
        try:
            vector = detect_imgs(path + "/" + fileName)
    
            # Kiểm tra việc trích xuất được không:
            if vector is not None:
                # Thêm vector vào dictionary
                data["data"].append(vector)
                data["label"].append(folder)
                print("compelted")
        except:
            # Bỏ qua nếu gặp lỗi:
            pass


compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted
compelted


In [50]:
data.keys()

dict_keys(['data', 'label'])

In [51]:
# Tạo series từ dictionary:
labelSeries = pd.Series(data['label'])

# Đếm số lần xuất hiện của mỗi nhãn:
labelCounts = labelSeries.value_counts()
print(labelCounts)

elonmusk    11
messi       10
Name: count, dtype: int64


In [54]:
# Save data -> file
Namefile = "./data/face_reg/data_face_feature.pickle"
pickle.dump(data,open(Namefile, mode = "wb"))